In [1]:
import csv
import requests
# import sys
from bs4 import BeautifulSoup

In [2]:
# 工作職稱與連接
def job(url):
    resp = requests.get(url)
    # 解析原始碼
    soup = BeautifulSoup( resp.text, "lxml" )
    # 尋找所有 class = "job-list-item" 的 "article" 標籤
    job_item = soup.find_all( "article" , "job-list-item" )
    
    item_list = []
    for i in job_item:
        # "div" & "a" 是標籤名
        tmp = i.find( "div" , "b-block__left" ).find("a")
        item_list.append({
            "title": tmp.getText().strip(),
            "link": "https:" + tmp.get("href") # get href 裡的文字
        })
    return item_list

In [3]:
# 獲取單個工作的詳細資訊
def job_info(job_url):
    resp_job = requests.get(job_url)
    soup = BeautifulSoup( resp_job.text , "lxml" )
    job = {}
    # rstrip():删除 string 結尾的指定字 , 默認為空格
#     job[ "Title" ] = soup.find_all("meta")[4]["content"].rstrip(" - 104人力銀行")
    job[ "Title" ] = soup.find_all("meta")[4]["content"].split("｜")[0]
    job["Content"] = soup.find("p").text.replace("\r"," ")
    job["Category"] = soup.find("dd" , "cate" ).text.strip().split()[0]
    
    tmp0 = soup.find( "dd" , "salary" ).text.strip().split()
    if "月薪" in tmp0:
         job["Salary"] = tmp0[1]
    elif "待遇面議" in tmp0:
        job["Salary"] = tmp0[0]
#     job["Salary"] = soup.find( "dd" , "salary" ).text.strip().split()
    job["Address"] = soup.find( "dd" , "addr" ).text.strip().split()[0]
    job["Tools"] = soup.select(".info")[1].find_all("dd")[-3].text
    job["Other"] = soup.select(".info")[1].find_all("dd")[-1].text
    
    return job

In [4]:
# 獲取所有工作詳細資訊的列表
def get_info(response):
    job_detail_list = []
    for x in response:
        job_detail = job_info(x["link"])
        job_detail_list.append(job_detail)
    
    if job_detail_list:
        return job_detail_list

In [5]:
# 輸出檔案
def export(data):
    file = "104_crawler.csv"
    col = [ "Title" , "Content" , "Category" , "Salary" , "Address" , "Tools" , "Other" ]
    try:
        # newline ="" 可解決輸出CSV檔多空一行的狀況
        with open(file , "w" , encoding="utf-8-sig", newline="" ) as f:
            csv104 = csv.DictWriter( f, fieldnames=col )
            csv104.writeheader()
            for i in data:
                csv104.writerow(i)
    except IOError:
        print("IO Error!")

In [6]:
job_detail_list = []
# 104 搜尋 " 資料分析 " 結果頁面
search_url = "https://www.104.com.tw/jobs/search/?ro=0&keyword=%E8%B3%87%E6%96%99%E5%88%86%E6%9E%90&order=15&asc=0&page="

for i in range(1,11):
    page_url = search_url + str(i) + "&jobexp=1&mode=s"
    response = job(page_url)
    job_detail = get_info(response)
    job_detail_list.extend(job_detail)
    
export(job_detail_list)
print("共印出筆數: ", len(job_detail_list))

共印出筆數:  200
